# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load in CSV
weather_data = pd.read_csv("../Weatherpy/Output/weather.csv")
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,77.72,74,80,7.74,PF,1618505679
1,Bethel,41.3712,-73.4140,50.00,96,100,6.13,US,1618505681
2,Arraial do Cabo,-22.9661,-42.0278,82.40,57,40,9.22,BR,1618505682
3,Cherskiy,68.7500,161.3000,12.20,95,100,5.28,RU,1618505683
4,Aykhal,66.0000,111.5000,14.86,96,88,8.57,RU,1618505685


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Lat", "Lng"]].astype(float)

# Store humidity for weight 
weights = weather_data["Humidity"].astype(float)

In [4]:
#Generate map with heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

# Show figure 
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
#Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow cities to max temp between 70 and 80 degrees, wind speed < 10mph, and zero cloudiness. 
narrowed_df = weather_data[(weather_data['Max Temp'] >= 70) & (weather_data['Max Temp'] <= 80)]
narrowed_df_2 = narrowed_df[(narrowed_df['Wind Speed'] < 10)]
narrowed_df_final = narrowed_df_2[(narrowed_df_2['Cloudiness'] == 0)]
narrowed_df_final

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,Neuquén,-38.9516,-68.0591,75.20,38,0,2.30,AR,1618505677
23,Saint-Philippe,-21.3585,55.7679,78.80,83,0,5.88,RE,1618505714
51,Necochea,-38.5473,-58.7368,75.58,50,0,9.17,AR,1618505754
231,Qiongshan,20.0058,110.3542,77.00,88,0,4.47,CN,1618506029
263,Inhambane,-23.8650,35.3833,78.01,73,0,6.55,MZ,1618506075
267,Ritchie,-29.0380,24.6028,75.20,29,0,2.30,ZA,1618506080
289,Umluj,25.0213,37.2685,76.33,65,0,9.04,SA,1618506114
498,Dolores,-36.3132,-57.6792,74.21,61,0,4.07,AR,1618506404
557,Saint-Joseph,-21.3667,55.6167,77.00,83,0,6.11,RE,1618506485


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
# Create hotel dataframe and add hotel name column
hotel_df = narrowed_df_final
hotel_df["Hotel Name"] = ''
hotel_df

C:\Users\lamcc\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,Neuquén,-38.9516,-68.0591,75.2,38,0,2.3,AR,1618505677,
23,Saint-Philippe,-21.3585,55.7679,78.8,83,0,5.88,RE,1618505714,
51,Necochea,-38.5473,-58.7368,75.58,50,0,9.17,AR,1618505754,
231,Qiongshan,20.0058,110.354,77,88,0,4.47,CN,1618506029,
263,Inhambane,-23.865,35.3833,78.01,73,0,6.55,MZ,1618506075,
267,Ritchie,-29.038,24.6028,75.2,29,0,2.3,ZA,1618506080,
289,Umluj,25.0213,37.2685,76.33,65,0,9.04,SA,1618506114,
498,Dolores,-36.3132,-57.6792,74.21,61,0,4.07,AR,1618506404,
557,Saint-Joseph,-21.3667,55.6167,77,83,0,6.11,RE,1618506485,
Hotel Name,,,,,,,,,,


In [33]:
# Base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Run a request to loop through rows and collect hotel name based on parameters
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params = {
        "location": f"{lat},{lng}",
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key
    }

    response = requests.get(base_url, params=params)
    hotels_data = response.json()
    
    try: 
        #print(results[0]['name'])
        hotel_df.loc[index,'Hotel Name'] = hotels_data['results'][0]['name']
    
    except:
        hotel_df.loc['Hotel Name'] = ""
        #print("No hotel available... skipping.")
    
hotel_df

C:\Users\lamcc\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\lamcc\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lamcc\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,Neuquén,-38.9516,-68.0591,75.2,38,0,2.3,AR,1618505677,Hotel Huemul
23,Saint-Philippe,-21.3585,55.7679,78.8,83,0,5.88,RE,1618505714,Les Embruns Du Baril
51,Necochea,-38.5473,-58.7368,75.58,50,0,9.17,AR,1618505754,Hotel Mirasol
231,Qiongshan,20.0058,110.354,77,88,0,4.47,CN,1618506029,Hilton Haikou
263,Inhambane,-23.865,35.3833,78.01,73,0,6.55,MZ,1618506075,Sentidos Beach Retreat
267,Ritchie,-29.038,24.6028,75.2,29,0,2.3,ZA,1618506080,The Cygnature Boutique Hotel
289,Umluj,25.0213,37.2685,76.33,65,0,9.04,SA,1618506114,HP Red Sea Hotel
498,Dolores,-36.3132,-57.6792,74.21,61,0,4.07,AR,1618506404,
557,Saint-Joseph,-21.3667,55.6167,77,83,0,6.11,RE,1618506485,Gandalf Safari Camp
Hotel Name,,,,,,,,,,


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Create hotel label layer
hotel_layer = gmaps.symbol_layer(locations,  info_box_content=[hotel_info])
    #locations, fill_color='rgba(0, 150, 0, 0.4)',
    #stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
   # info_box_content=[hotel_info]
#)

fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig

IndexError: list index out of range

In [ ]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig